In [1]:
import os

os.chdir("..")
print(f"Changed working directory to: {os.getcwd()}")

Changed working directory to: /home/jovyan/work/FlareSense


In [2]:
import torch
import mlflow
import dagshub
import itertools
import torchmetrics
import src.utils.data15min as data
import pytorch_lightning as pl

from huggingface_hub import snapshot_download
from torchvision import transforms
from tqdm.notebook import tqdm
from src.models.EfficientNetV2SBinaryClassifier import EfficientNetV2SBinaryClassifier

mlflow.pytorch.autolog()
torch.set_float32_matmul_precision("high")

2024/01/05 12:00:40 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
2024-01-05 12:00:40.998947: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-05 12:00:41.019607: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 12:00:41.019628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 12:00:41.020360: E external/local_xl

In [3]:
DATA_FOLDER_PATH = "data/raw/exported/"
INSTRUMENTS = ["Australia-ASSA_02", "Australia-ASSA_62"]
NUM_WORKERS = 12

In [4]:
# download needed files
snapshot_download(
    "StellarMilk/ecallisto-bursts",
    repo_type="dataset",
    allow_patterns=[f"{instrument}.zip" for instrument in INSTRUMENTS] + ["metadata.csv"],
    local_dir=DATA_FOLDER_PATH,
    revision="main",
)

for instrument in INSTRUMENTS:
    # if data available, skip
    if os.path.exists(f"{DATA_FOLDER_PATH}{instrument}"):
        print(f"Skipping {instrument}")
        continue
    
    # unzip if needed
    print(f"Unzipping {instrument}")
    !unzip -q {DATA_FOLDER_PATH}{instrument}.zip -d {DATA_FOLDER_PATH}

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Skipping Australia-ASSA_02
Skipping Australia-ASSA_62


In [ ]:
lst_lr = [1e-4]
lst_weight_decay = [1e-3]

data_module = data.ECallistoDataModule(
    data_folder=DATA_FOLDER_PATH,
    batch_size=64,
    num_workers=NUM_WORKERS,
    val_ratio=0.15,
    test_ratio=0.15,
    img_size=(224, 224),
    use_augmented_data=True,
    filter_instruments=["Australia-ASSA_02", "Australia-ASSA_62"],
    seed=0,
)
data_module.setup()

for current_lr, current_weight_decay in itertools.product(lst_lr, lst_weight_decay):
    model = EfficientNetV2SBinaryClassifier(lr=current_lr, weight_decay=current_weight_decay)

    dagshub.init("FlareSense", "FlareSense", mlflow=True)
    mlflow.start_run()

    mlflow.log_params({
        "model": "EfficientNetV2",
        "batch_size": data_module.batch_size,               
        "val_ratio": data_module.val_ratio,
        "test_ratio": data_module.test_ratio,
        "use_data_augmentation": data_module.use_augmented_data,
        "filter_instruments": data_module.filter_instruments,
    })

    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_{run_id}")

    trainer = pl.Trainer(max_epochs=50, log_every_n_steps=1)

    trainer.fit(
        model,
        train_dataloaders=data_module.train_dataloader(),
        val_dataloaders=data_module.val_dataloader(),
    )

    trainer.test(model, dataloaders=data_module.test_dataloader())

    mlflow.end_run()

Repository initialized!

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024/01/05 12:00:44 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.11/site-packages/mlflow/pytorch/_lightning_autolog.py:356: UserWarning: Autologging is known to be compatible with pytorch-lightning versions between 1.4.9 and 2.1.2 and may not succeed with packages outside this range."


Run ID: 9d4c17a9a4904c549db81886b248e9b9
Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_9d4c17a9a4904c549db81886b248e9b9


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | precision         | BinaryPrecision | 0     
1 | recall            | BinaryRecall    | 0     
2 | efficientnet_v2_s | EfficientNet    | 20.2 M
------------------------------------------------------
18.4 M    Trainable params
1.8 M     Non-trainable params
20.2 M    Total params
80.715    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]